### Imports

In [1]:
from ndex2.nice_cx_network import NiceCXNetwork
import ndex2.client as nc
import ndex2
import networkx as nx
import pandas as pd
import os
import sys

In [2]:
 def nice_cx_to_node_dataframe(nice_cx_network):
    """
    Create a Pandas DataFrame in which each row is a node and columns are node attributes.
     Example:
        ``df = nice_cx_to_node_dataframe(my_nice_cx_network) # df is now a pandas dataframe``
    :return: Pandas dataframe
    :rtype: Pandas dataframe
    """
    #TODO expand documentation
    rows = []
    node_items = None
    if sys.version_info.major == 3:
        node_items = nice_cx_network.nodes.items()
    else:
        node_items = nice_cx_network.nodes.iteritems()
    # v is the node item values
    # k is the index of the node
    attribute_names = {"name", "node_id"}
    for node_id, v in node_items:
        row = {}
        row["name"] = v.get('n')
        row["node_id"] = node_id
        for attribute in nice_cx_network.get_node_attributes(node_id):
            attribute_names.add(attribute.get('n'))
            row[attribute.get('n')] = attribute.get('v')
        rows.append(row)
    df_columns = list(attribute_names)
    return_df = pd.DataFrame(rows, columns=df_columns)
    return return_df

def jaccard_similarity_lists(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

def jaccard_similarity_sets(set1, set2):
    intersection = len(list(set1.intersection(set2)))
    union = (len(set1) + len(set2)) - intersection
    return float(intersection) / union

### The NDEx Account

In [3]:
my_account="dexterpratt"
my_password="cytoscaperules"
my_server="http://public.ndexbio.org"

if my_account == 'enter your username here':
    print('*******WARNING!!!!*******')
    print('Please change the username and password before proceeding')
else:
    try:
        my_ndex=nc.Ndex2(my_server, my_account, my_password)
        my_ndex.update_status()
        print("Success.  Please continue.")
    except Exception as inst:
        print("Could not access account %s with password %s" % (my_account, my_password))
        print(inst.args)

Success.  Please continue.


### The COVID-19 - Human Interactors Gene Set

In [4]:
interactor_set = {"F2RL1", "ADAM9", "TOR1A", "PMPCA", "CISD3", "ATP13A3", "MPHOSPH10", "SLC27A2", "TIMM10B", "GORASP1", "GNG5", "EXOSC5", "EMC1", "NPC2", "PPT1", "AGPS", "LMAN2", "CDK5RAP2", "PLD3", "NUP98", "PLOD2", "NPTX1", "GCC1", "RBM28", "LARP1", "REEP5", "MDN1", "TBK1", "TM2D3", "CLCC1", "FAM8A1", "PTBP2", "POR", "RAB5C", "GTF2F2", "GRIPAP1", "VPS39", "NEU1", "MOGS", "GOLGB1", "PLAT", "AKAP9", "PTGES2", "DPY19L1", "SLC30A9", "PDE4DIP", "HS6ST2", "RNF41", "FBN2", "G3BP2", "NDFIP2", "VPS11", "BRD4", "C1orf50", "NUP210", "PIGO", "IMPDH2", "GPAA1", "GDF15", "PIGS", "ERO1B", "ZNF318", "WASHC4", "AP2A2", "ARL6IP6", "NGLY1", "ACSL3", "TOMM70", "EDEM3", "LOX", "PDZD11", "RAB18", "NARS2", "NUP88", "TARS2", "FKBP15", "INTS4", "AP3B1", "ERGIC1", "NINL", "GGCX", "SLC9A3R1", "BCKDK", "NOL10", "CHPF", "PITRM1", "CIT", "PCSK6", "TYSND1", "CSNK2B", "ATE1", "MRPS5", "DNAJC19", "JAKMIP1", "GFER", "WFS1", "SLC44A2", "UGGT2", "NUTF2", "MRPS27", "ALG11", "FOXRED2", "PVR", "ERP44", "CEP350", "DNAJC11", "SIRT5", "STOM", "NLRX1", "CYB5B", "USP54", "FAR2", "CUL2", "RAB2A", "ZNF503", "PRIM1", "IL17RA", "ITGB1", "EIF4E2", "TLE3", "DDX10", "DCAF7", "CEP43", "ANO6", "RBM41", "SBNO1", "ZDHHC5", "CYB5R3", "SLU7", "PPIL3", "UPF1", "SRP19", "TIMM9", "NUP62", "ATP5MG", "CEP250", "MTCH1", "ALG8", "DDX21", "PABPC1", "FBXL12", "BRD2", "UBXN8", "SLC30A6", "ACAD9", "RAB8A", "ELOB", "PLEKHF2", "IDE", "LARP7", "NEK9", "DNMT1", "PRKAR2B", "NAT14", "EXOSC2", "GRPEL1", "HEATR3", "PCNT", "GCC2", "SCAP", "TLE1", "BZW2", "UBAP2", "SELENOS", "AKAP8", "CLIP4", "SEPSECS", "HS2ST1", "CRTC3", "RIPK1", "RBX1", "GOLGA2", "FBN1", "ATP1B1", "NDUFAF2", "ETFA", "PABPC4", "MIB1", "RETREG3", "SMOC1", "HDAC2", "AATF", "MARK2", "CENPF", "FBLN5", "NSD2", "MRPS25", "RPL36", "PMPCB", "GGH", "GPX1", "NGDN", "QSOX2", "AP2M1", "PRIM2", "TUBGCP2", "COQ8B", "REEP6", "ERMP1", "RAB1A", "MFGE8", "RDX", "TBKBP1", "TCF12", "ERC1", "RAB10", "FAM98A", "ZC3H7A", "ABCC1", "NDUFB9", "GOLGA7", "PUSL1", "RAB7A", "SPART", "ACADM", "RAE1", "TBCA", "PLEKHA5", "NDUFAF1", "NIN", "TLE5", "COL6A1", "DPH5", "PRKAR2A", "MARK3", "MYCBP2", "TOR1AIP1", "HMOX1", "DCTPP1", "BCS1L", "ADAMTS1", "ERLEC1", "TIMM29", "NUP214", "NUP58", "MRPS2", "MARK1", "SIL1", "POLA2", "CHMP2A", "ARF6", "SUN2", "POLA1", "EIF4H", "FASTKD5", "ATP6V1A", "TAPT1", "TMEM97", "FKBP7", "STOML2", "STC2", "CEP135", "CEP112", "SRP54", "SDF2", "SNIP1", "AKAP8L", "EXOSC3", "RAB14", "ZC3H18", "TMEM39B", "CNTRL", "OS9", "ATP6AP1", "LARP4B", "FAM162A", "TIMM8B", "NUP54", "AAR2", "THTPA", "SLC30A7", "PRRC2B", "FYCO1", "PKP2", "DCAKD", "YIF1A", "GIGYF2", "GHITM", "CSDE1", "TRIM59", "HYOU1", "BAG5", "HSBP1", "SIGMAR1", "TRMT1", "MOV10", "SLC25A21", "INHBE", "GLA", "COMT", "UBAP2L", "MTARC1", "PSMD8", "FKBP10", "PRKACA", "EXOSC8", "ZYG11B", "RAP1GDS1", "ELOC", "MAT2B", "POFUT1", "POGLUT2", "HECTD1", "GOLGA3", "SAAL1", "TMED5", "MIPOL1", "RHOA", "CCDC86", "CEP68", "ECSIT"}

### The Hierarchy

In [16]:
hierarchy_uuid = "3f3f10a8-75d6-11ea-aaef-0ac135e8bacf" # diffusion
# hierarchy_uuid = "b7c7d89e-75d6-11ea-aaef-0ac135e8bacf" # neighborhood
# hierarchy_uuid = "8639b05b-75d6-11ea-aaef-0ac135e8bacf" #interconnect
hierarchy_network = ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid=hierarchy_uuid, 
                                                    password=my_password, username=my_account)
# append "interactors" to its name to indicate that interactor annotations have been addded
hierarchy_network.set_name(hierarchy_network.get_name() + " - interactors")
# add the interactors as a network attribute
hierarchy_network.set_network_attribute("COVID-19-interactors", list(interactor_set), type="list_of_string")

In [17]:
#node_items = hierarchy_network.nodes.items()
#node_items
node_df = nice_cx_to_node_dataframe(hierarchy_network)
node_df

,CD_AnnotatedMembers,CD_AnnotatedMembers_SourceDB,CD_AnnotatedMembers_Pvalue,name,CD_MemberList_LogSize,CD_AnnotatedMembers_Overlap,CD_Labeled,CD_MemberList,node_id,CD_AnnotatedAlgorithm,CD_AnnotatedMembers_Size,CD_MemberList_Size,CD_AnnotatedMembers_SourceTerm,CD_CommunityName,CD_NonAnnotatedMembers
0,NaN,NaN,NaN,C1805,2.0,0.0,false,KCNS3 SLC44A2 STUM TAPT1,172193,Annotated by gProfiler [Docker: coleslawndex/c...,0,4,NaN,(none),NaN
1,NaN,NaN,NaN,C1804,2.585,0.0,false,ATP13A3 C5AR1 CTAGE7P PIGO TMEM63A ZDHHC12,172192,Annotated by gProfiler [Docker: coleslawndex/c...,0,6,NaN,(none),NaN
2,NaN,NaN,NaN,C1803,2.0,0.0,false,NDC1 SCAP YIF1A ZNRF4,172191,Annotated by gProfiler [Docker: coleslawndex/c...,0,4,NaN,(none),NaN
3,NaN,NaN,NaN,C1802,2.0,0.0,false,ADAM10 ADAM33 FBXO2 ITGB5,172190,Annotated by gProfiler [Docker: coleslawndex/c...,0,4,NaN,(none),NaN
4,NaN,NaN,NaN,C1801,2.0,0.0,false,AKAP14 MAP2 PRKACG PRKAR2A,172189,Annotated by gProfiler [Docker: coleslawndex/c...,0,4,NaN,(none),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,A4GNT B3GNT3 B4GALT3 B4GALT4 B4GALT7 CHPF CHPF...,GO:BP,1.6100509655313828E-40,C1468,7.622,0.067,true,A4GNT ACOT11 ADAM10 ADAM11 ADAM32 ADAM33 ADAM9...,171856,Annotated by gProfiler [Docker: coleslawndex/c...,38,197,GO:0009100,glycoprotein metabolic process,ACOT11 ADAM10 ADAM11 ADAM32 ADAM33 ADAM9 ADAMT...
338,ABCD1 AGK B4GAT1 CHST12 COA1 CSGALNACT2 DERL2 ...,GO:CC,1.0568428010354981E-38,C1467,8.439,0.061,true,ABCC1 ABCD1 ABHD5 ACAD10 ACVR2B ADGRE5 ADGRG5 ...,171855,Annotated by gProfiler [Docker: coleslawndex/c...,43,347,GO:0031300,intrinsic component of organelle membrane,ABCC1 ABHD5 ACAD10 ACVR2B ADGRE5 ADGRG5 ADGRL3...
339,NaN,NaN,NaN,C1466,8.5,0.0,false,AAGAB AATF ABHD15 ABL2 ACADL ACER3 ADAD2 ADCYA...,171854,Annotated by gProfiler [Docker: coleslawndex/c...,0,362,NaN,(none),NaN
340,A4GNT ABCD1 ACAD9 ACADL ACER3 ACSL3 ADAM10 ADG...,GO:CC,5.299425872799433E-211,C1465,10.359,0.1,true,A4GNT AAGAB AASS AATF ABCA6 ABCC1 ABCD1 ABCF2 ...,171853,Annotated by gProfiler [Docker: coleslawndex/c...,437,1313,GO:0031090,organelle membrane,AAGAB AASS AATF ABCA6 ABCC1 ABCF2 ABHD15 ABHD5...


### Scoring

In [18]:
def in_community(row, interactors):
    #print(len(row.get("CD_MemberList")))
    #result = list(set(row.get("CD_MemberList")).intersection(interactors))
    #if (len(result) > 0):
    #    print(result)
    return list(set(row.get("CD_MemberList").split()).intersection(interactors))

def community_overlap(row, interactors):
    intersection = len(set(row.get("CD_MemberList").split()).intersection(interactors))
    memberlist_size = int(row.get("CD_MemberList_Size"))
    if isinstance(memberlist_size, (int, float)):
        return intersection / memberlist_size
    else:
        return 0

def jacquard_interactors(row, interactors):
    return jaccard_similarity_sets(set(row.get("CD_MemberList").split()), interactors)
                            
node_df["CD_InteractorMembers"] = node_df.apply(lambda row: in_community(row, interactor_set), axis=1)
                            
node_df["CD_InteractorMemberOverlap"] = node_df.apply(lambda row: community_overlap(row, interactor_set), axis=1)
node_df

# append columns + name them

,CD_AnnotatedMembers,CD_AnnotatedMembers_SourceDB,CD_AnnotatedMembers_Pvalue,name,CD_MemberList_LogSize,CD_AnnotatedMembers_Overlap,CD_Labeled,CD_MemberList,node_id,CD_AnnotatedAlgorithm,CD_AnnotatedMembers_Size,CD_MemberList_Size,CD_AnnotatedMembers_SourceTerm,CD_CommunityName,CD_NonAnnotatedMembers,CD_InteractorMembers,CD_InteractorMemberOverlap
0,NaN,NaN,NaN,C1805,2.0,0.0,false,KCNS3 SLC44A2 STUM TAPT1,172193,Annotated by gProfiler [Docker: coleslawndex/c...,0,4,NaN,(none),NaN,"[SLC44A2, TAPT1]",0.500000
1,NaN,NaN,NaN,C1804,2.585,0.0,false,ATP13A3 C5AR1 CTAGE7P PIGO TMEM63A ZDHHC12,172192,Annotated by gProfiler [Docker: coleslawndex/c...,0,6,NaN,(none),NaN,"[PIGO, ATP13A3]",0.333333
2,NaN,NaN,NaN,C1803,2.0,0.0,false,NDC1 SCAP YIF1A ZNRF4,172191,Annotated by gProfiler [Docker: coleslawndex/c...,0,4,NaN,(none),NaN,"[YIF1A, SCAP]",0.500000
3,NaN,NaN,NaN,C1802,2.0,0.0,false,ADAM10 ADAM33 FBXO2 ITGB5,172190,Annotated by gProfiler [Docker: coleslawndex/c...,0,4,NaN,(none),NaN,[],0.000000
4,NaN,NaN,NaN,C1801,2.0,0.0,false,AKAP14 MAP2 PRKACG PRKAR2A,172189,Annotated by gProfiler [Docker: coleslawndex/c...,0,4,NaN,(none),NaN,[PRKAR2A],0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,A4GNT B3GNT3 B4GALT3 B4GALT4 B4GALT7 CHPF CHPF...,GO:BP,1.6100509655313828E-40,C1468,7.622,0.067,true,A4GNT ACOT11 ADAM10 ADAM11 ADAM32 ADAM33 ADAM9...,171856,Annotated by gProfiler [Docker: coleslawndex/c...,38,197,GO:0009100,glycoprotein metabolic process,ACOT11 ADAM10 ADAM11 ADAM32 ADAM33 ADAM9 ADAMT...,"[ADAM9, GGH, PLAT, HYOU1, SIL1, OS9, FOXRED2, ...",0.162437
338,ABCD1 AGK B4GAT1 CHST12 COA1 CSGALNACT2 DERL2 ...,GO:CC,1.0568428010354981E-38,C1467,8.439,0.061,true,ABCC1 ABCD1 ABHD5 ACAD10 ACVR2B ADGRE5 ADGRG5 ...,171855,Annotated by gProfiler [Docker: coleslawndex/c...,43,347,GO:0031300,intrinsic component of organelle membrane,ABCC1 ABHD5 ACAD10 ACVR2B ADGRE5 ADGRG5 ADGRL3...,"[MTCH1, ABCC1, WFS1, QSOX2, SLC44A2, MDN1, ATP...",0.187320
339,NaN,NaN,NaN,C1466,8.5,0.0,false,AAGAB AATF ABHD15 ABL2 ACADL ACER3 ADAD2 ADCYA...,171854,Annotated by gProfiler [Docker: coleslawndex/c...,0,362,NaN,(none),NaN,"[PITRM1, GOLGA3, NINL, CEP68, NUP62, ARL6IP6, ...",0.226519
340,A4GNT ABCD1 ACAD9 ACADL ACER3 ACSL3 ADAM10 ADG...,GO:CC,5.299425872799433E-211,C1465,10.359,0.1,true,A4GNT AAGAB AASS AATF ABCA6 ABCC1 ABCD1 ABCF2 ...,171853,Annotated by gProfiler [Docker: coleslawndex/c...,437,1313,GO:0031090,organelle membrane,AAGAB AASS AATF ABCA6 ABCC1 ABCF2 ABHD15 ABHD5...,"[PITRM1, FAM98A, WFS1, SLC25A21, ATP6V1A, DCTP...",0.198781


### Add New Attributes to Hierarchy

In [8]:
# for each of the new columns, iterate setting node attributes
for index, row in node_df.iterrows(): 
    hierarchy_network.set_node_attribute(row["node_id"],
                                         "CD_InteractorMembers", 
                                         row["CD_InteractorMembers"], 
                                         type="list_of_string" )
    hierarchy_network.set_node_attribute(row["node_id"],
                                         "CD_InteractorMemberOverlap", 
                                         row["CD_InteractorMemberOverlap"], 
                                         type="double" )


### Save to NDEx

In [9]:
if my_account == 'enter your username here':
    print('*******WARNING!!!!*******')
    print('Please change the username and password before proceeding')
else:
    upload_message = hierarchy_network.upload_to(my_server, my_account, my_password)
    print(upload_message)

Generating CX
http://public.ndexbio.org/v2/network/6b3ea430-7769-11ea-aaef-0ac135e8bacf


In [19]:
node_df.to_excel(hierarchy_network.get_name() + " nodes.xlsx")